----
**Research Question**

## Investigation of changes in **Depressive-Symptoms** influenced by drinking frequency of **Sugar-Sweetened** soft drinks.
----

In [108]:
# importting necessary libraries 

import pandas as pd
import numpy as np
import yaml as yam
from bokeh.plotting import figure,show,output_file
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.palettes import Category10
import holoviews as hv
from bokeh.io import output_notebook

output_notebook()


Loading BokehJS ...

----
## Part 1 : Loading The Data
----

In [109]:
# creating a config file , to read data from

def get_config():
    with open('config.yaml','r') as datasource:
        config = yam.safe_load(datasource)
        return config

config = get_config()

depressive_symptoms = config['depressive_symptoms']
frequency_of_drinking = config['frequency_of_drinking']

# reading data by creating pandas dataframe
depsymp = pd.read_csv(depressive_symptoms)
drinkfreq = pd.read_csv(frequency_of_drinking)



----
## Part 2 : **Data Preparation**
---- 

* Renaming columns

In [110]:
# Renaming the desired columns in order to have a better understanding of the columns name.

def rename(df,col_list) :
    rename = df.rename(columns= col_list)
    df = rename
    return df

depsymp_col_list = {'isced11':'Edu_Level','hlth_pb':'Health_Issue',
            'sex':'Gender','age':'Age_Range','geo':'Country','TIME_PERIOD':'Time_Period',
            'OBS_VALUE':'Obs_Value'}

depsymp = rename(depsymp,depsymp_col_list)


drinkfreq_col_list = {'frequenc':'Drink_Freq','sex':'Gender','isced11':'Edu_Level',
            'age':'Age_Range','geo':'Country','TIME_PERIOD':'Time_Period','OBS_VALUE':'Obs_Value'}

drinkfreq = rename(drinkfreq,drinkfreq_col_list)

* Identifying missing values

In [111]:
# defining a function to return missing values in multiple dataframes,
# output in the form of a dictionary will contain both dataframe and columns 
# with missing values 

# depsymp.isna().sum()
# depsymp.isnull().sum()


def find_missing_values(df_list):
    missing_values = []
    for i,df in enumerate(df_list):

        df_missing = df.isna().sum().to_dict()
        df_missing = {k: v for k,v in df_missing.items() if v > 0}
        missing_values.append(f'dataframe {i+1} = {df_missing}')
                
    return missing_values


missing_values = find_missing_values([depsymp,drinkfreq])

# print(missing_values)

* Identifying unreliable observations

In [112]:
# Flag columns indicate observations which are not reliable.
# Below, identification of the specific value for unreliable abservations and the length of rows containing 
# them where inspected.

print(depsymp['OBS_FLAG'].unique())
print(drinkfreq['OBS_FLAG'].unique())
print(len(depsymp[depsymp['OBS_FLAG'] == 'u']))
print(len(drinkfreq[drinkfreq['OBS_FLAG'] == 'u']))

[nan 'u']
[nan 'u']
36
27


* Removing unreliable observations and columns with missing values

In [113]:
# defining a function that takes in list of dataframes and returns 
# cleaned dataframes (without columns containing missing values, and rows with unreliable observations)

def drop_missing_values(dataframes):
    cleaned_dataframes = []
    for df in dataframes:
        if 'OBS_FLAG' in df.columns :
            df = df[df['OBS_FLAG'] != 'u']
        df_dropped = df.dropna(axis=1)
        cleaned_dataframes.append(df_dropped)
    return cleaned_dataframes

list = [depsymp,drinkfreq]
cleaned_dataframes = drop_missing_values(list)

depsymp = cleaned_dataframes[0]
drinkfreq = cleaned_dataframes[1]
# drinkfreq



* Dropping irrelevant columns 

In [114]:
# dropping irrelevent columns using if statements to 
# avoid code execution more than once

if 'DATAFLOW' in depsymp.columns :
    depsymp = depsymp.drop(depsymp.columns[[0,1,2,3,4]],axis=1)

if 'DATAFLOW' in drinkfreq.columns :
    drinkfreq = drinkfreq.drop(drinkfreq.columns[[0,1,2,3,7]],axis=1)
    
# print(depsymp)
# print(drinkfreq) 


* Inspect duplicated observations

In [115]:
# checking for duplicated values in both dataframes.

# df.duplicated()
# df.drop_duplicates()

def check_for_duplicated(df_list) :
    for df in df_list :
        if df.duplicated().any() :
            return True
        else :
            return False

duplicated = check_for_duplicated([depsymp,drinkfreq])
# print(duplicated)


----
## Part 3 : Reshaping the data
----

* Merging data frames (Due to creation of redundancy "in the form of duplication"
   which makes it impossible to further process the data this part was canceled.)

In [116]:
## using pandas merge method we are going to combine two dataframes 
## into one with values of interest for further analysing.

# df = pd.merge(drinkfreq,depsymp,
#                 on=['Country','Time_Period','Age_Range','Gender'],
#                 suffixes=['_Dri','_Dep'], #especific suffixes helps to identify the coulmn after being merged
#                 how='inner')

## df = df.sort_values(by=['Obs_Value_Dri','Obs_Value_Dep'], ascending= False).copy()

* Age_range column contains multiple ranges, in the following function 
   we try to keep those ranges which have more coverage.

In [117]:
# # inspection of unique values in order to keep the desired ones in next step
# print(depsymp['Age_Range'].unique() )
# print(drinkfreq['Age_Range'].unique())

# Filtering the data frame for the desired values

def filter_age_range(df_list) :
    filtered_age_range = []

    for df in df_list :
        df = df[df['Age_Range'].isin(['Y15-24','Y25-34','Y35-44','Y45-54','Y55-64','Y65-74'])].copy()
        filtered_age_range.append(df)
    return filtered_age_range


depsymp = filter_age_range([depsymp,drinkfreq])[0]
drinkfreq = filter_age_range([depsymp,drinkfreq])[1]


* match_maker() is a function that creates a list of tuples containing paired unique values 
   of the two provided column from our desired data frame, makes it easier to groupby in the next step.

In [118]:
# we will define a function to make a list containing 
# tuples of paired values of any two given columns.

def match_maker(df,col_1,col_2) :

    result = []

    gen_uni = df[col_1].unique()
    drink_uni = df[col_2].unique()
    for x in gen_uni :
        for y in drink_uni :
            if (x,y) not in result :
                result.append((x,y))

    return result
    

# match_maker(drinkfreq,'Gender','Drink_Freq')

* data_grouper(), this function is designed to access the specific values 
   inside a group and grouping them again in order to make plotting more focused. Actually our final dataframes 
   are going to be made here inside this function.

In [119]:
def data_grouper(df,col_1,col_2) :

    grouped_data = []
    for i in match_maker(df,col_1,col_2) :
        grouping = df.groupby([col_1,col_2]).get_group(i)
        grouped_data.append(grouping)
    return grouped_data

# In total we will have 21 data frames, which are distinguished from each other by their Gender
# our 21 data frames are consisting female values, male values , and total values which include both Gender.
# the following data frames are those which contain total values for both Gender.  


upto3w_tot     = data_grouper(drinkfreq,'Gender','Drink_Freq')[8]
fourto6w_tot   = data_grouper(drinkfreq,'Gender','Drink_Freq')[9]
gen1perday_tot = data_grouper(drinkfreq,'Gender','Drink_Freq')[10]
nevorocc_tot   = data_grouper(drinkfreq,'Gender','Drink_Freq')[11]
dpr_tot        = data_grouper(drinkfreq,'Gender','Drink_Freq')[6]
dpr_mjr_tot    = data_grouper(drinkfreq,'Gender','Drink_Freq')[7]
dpr_oth_tot    = data_grouper(drinkfreq,'Gender','Drink_Freq')[8]

# upto3w_tot

# match_maker(depsymp,'Gender','Health_Issue')
# match_maker(drinkfreq,'Gender','Drink_Freq')

* Age_filter

In [120]:
# Since we are to investigate the changes in different age ranges , 
# first we need to filter our data frames on their age column values, which consists 6 ranges.

def age_filter(df) :

    filtered_age = []

    age_unique = df['Age_Range'].unique()

    for age in age_unique :
        age_filter = df.loc[df['Age_Range'] == age]
        
        filtered_age.append(age_filter)

        # print(len(filtered_age),end=',')
        # print(filtered_age[0])

    return filtered_age

# filtered_age = age_filter(upto3w_tot)

----
## Part 4 : Smooth The Data (on going)
----

In [121]:
# defining a function that takes in a list of dataframes and retunrs a list of 
# smoothed data frames. The built in method '.rolling().mean()' was used.

# def smooth_dataframes(dataframes,window_size) :
#     smoothed_dataframes = []
#     for df in dataframes :
#         df = df.copy()
#         df['Obs_Value_Dri'] = df['Obs_Value_Dri'].rolling(window=window_size).mean()
#         smoothed_dataframes.append(df)
#     return smoothed_dataframes

# dataframes = smooth_dataframes([df1,df2,df3,df4],3)
# dataframes[1]

----
## Part 5 : Data Visualization (on going)
----

* Line plot

In [122]:
# The purpose of this function is to plot line graphs of multiple data frames which are one by one 
# matched on their age column value, in this way we are able to track the changes in both data frames
# in regard with each other and comparing would be facilated.

def make_line_plot(df1,df2) :

    a = age_filter(df1)
    b = age_filter(df2)
    plot_list = []
    
    # zip() function takes in two list of data frames and returns, a tuple with paired data frames with 
    # similar index in the two original lists.
    
    for i,j in zip(a,b) : 
        l_fig = figure(x_range =i['Country'],
                title=f"Consuming Sweetened Drinks/ Depressive Symptoms Between ages {i['Age_Range'].iloc[1]}",
                x_axis_label='EU Countries', y_axis_label='Frequency in Percentage',
                        plot_height=300,plot_width=550)
        
        l_fig.line(x=i['Country'],y=i['Obs_Value'],color='#CC79A7',
                #    line_dash='dashed',
                   line_width=1.2)
        l_fig.line(x=j['Country'],y=j['Obs_Value'],color='#0072B2',
                #    line_dash='dashed',
                   line_width=1.2)
        
        # l_fig.xgrid.grid_line_color = 'red'
        l_fig.xgrid.grid_line_alpha = 0.3
        l_fig.xaxis.axis_line_alpha = 0
        l_fig.ygrid.grid_line_alpha = 1

        plot_list.append(l_fig)
        # print(plot_list)
    
    
    grid = gridplot(plot_list,ncols=2)
    
    output_file('myplot.html')
    show(grid)

make_line_plot(upto3w_tot,dpr_tot)

In [123]:

def make_plot(df_list,title,y,y_label) :

    plot_list = []
    
    for i,df in enumerate(df_list) :

        fig = figure(x_range= df.Country.unique(), title=title[i],
                    plot_height = 300, plot_width = 600,
                    x_axis_label='EU Countries', y_axis_label= y_label)
        fig.y_range.start = 0
        fig.xgrid.grid_line_color = None
        fig.ygrid.grid_line_alpha = 1

        fig.vbar(x=df['Country'], top=df[y] , width= 0.8)

        plot_list.append(fig)
                     
    grid = gridplot(plot_list, ncols= 2)

    output_file('myplot.html')

    show(grid)



In [124]:
# df_list = [A,B,C,D]

title_list = ['1-3W  Drinking / Country Percentage',
              '4-6W  Drinking / Country Percentage',
              'GE1D  Drinking / Country Percentage',
              'NVR_OCC  Drinking / Country Percentage']

# make_plot(df_list,title_list,'Obs_Value_Dri','DRI Percentage')

In [125]:

# df_list = [df1,df2,df3,df4]
title_list1 = ['(DPR) Depressive symptoms / Country ratio in Percentage',
               '(DPR_MJR) Major Depressive symptoms / Country ratio in Percentage',
               '(DPR_OTH) Other Depressive symptoms / Country ratio in Percentage',
               '1']

# make_plot(df_list,title_list1,'Obs_Value_Dep','DEP percentage')
